# Catalog generators

**Caden Gobat**, The George Washington University

In [1]:
import pandas as pd
from code.utilities import split_filters
from code.xrt import XRT_lightcurve, get_photonIndex, get_temporalIndex, get_columnDensity

alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
alpha += alpha.lower()
numeric = ".0123456789"

## Short GRB sample

In [2]:
swift = pd.read_html("https://swift.gsfc.nasa.gov/archive/grb_table/fullview/")[0] # get latest Swift catalog
swift.columns = [col[0] for col in swift.columns] # reduce/flatten MultiIndex
for grb in swift[swift["TriggerNumber"].apply(pd.to_numeric,errors="coerce").isna()]["GRB"]: # non-Swift bursts
    bat_data = pd.read_html(f"https://swift.gsfc.nasa.gov/archive/grb_table/fullview/{grb}/")[0]
    T90 = bat_data.loc[bat_data[0]=="T90: c",1].values # get the T90 anyway, if it exists
    swift.loc[swift["GRB"]==grb,"BAT T90[sec]"] = pd.to_numeric(T90, errors="coerce")

In [3]:
GCN_flagged = ['050724','050813','051210','060121','060313',
         '060427','060801','061006','061201','061210','061217',
         '070707','070729','070809','071227','080123','080503',
         '080913','080919','081024A','081105','081211','081211B','081226A',
         '090423','090426','090510','090515','090531B','090607','090621A',
         '090621B','091109B','100117A','100206A','100213A','100625A','100628A',
         '100702A','100724A','100816A','101219A','101224A','110106A','110112A',
         '110402A','110715A','110802A','111020A','111117A','111121A','111222A',
         '120305A','120521A','120630A','120804A','121226A','130313A','130515A',
         '130603B','130716A','130822A','130912A','131002A','131004A','140129B',
         '140209A','140320A','140516A','140611A','140622A','140903A','140930B',
         '141212A','150101A','150120A','150301A','150423A','150424A','150728A',
         '150831A','151127A','151229A','160228A','160303A','160408A','160410A',
         '160411A','160425A','160601A','160624A','160821B','160927A','161001A',
         '161004A','161104A','161129A','170127B','170428A','170524A','170728A',
         '170728B','170822A','170827A','171007A','171211A','180204A','180402A',
         '180418A','180618A','180727A','180805B','181123B','181126A','190627A',
         '190719C','191031C','191031D','200219A','200409A','200411A','200512A',
         '200517A','200522A','200716C','200729A','200826A','200907B','200917A',
         '201006A','201015A','201221A','201221D','210217A','210323A','210410A',
         '210618A','210704A','210708A','210725B','210726A','210919A','211023B',
         '211106A','211207A','211221A','211227A'] # flagged GCNs w/ XRT observations

published = ['050202', '050509B', '050709', '050724A', '050813', '050906', '050925',
          '051210', '051221A', '060121', '060313', '060502B', '060801', '061006',
          '061201', '061210', '061217', '070209', '070406', '070429B', '070707',
          '070714B', '070724A', '070729', '070809', '070810B', '071017', '071112B',
          '071227', '080121', '080123', '080426', '080503', '080702A', '080905A',
          '080919', '081024A', '081024B', '081226A', '081226B', '090305', '090305A',
          '090426', '090426A', '090510', '090515', '090607', '090621B', '090916',
          '091109B', '091117', '100117A', '100206A', '100213', '100625A', '100628A',
          '100702A', '101219A', '101224A', '110112A', '110112B', '110420B', '111020A',
          '111117A', '111121A', '111222A', '120229A', '120305A', '120521A', '120630A',
          '120804A', '120817B', '121226A', '130313A', '130515A', '130603B', '130626A',
          '130716A', '130822A', '130912A', '131004A', '131125A', '131126A', '131224A',
          '140129B', '140320A', '140402A', '140414A', '140516A', '140606A', '140619B',
          '140622A', '140903A', '140930B', '141202A', '141205A', '141212A', '150101A',
          '150101B', '150120A', '150301A'] # Fong, et al.

published += ['130313A', '130822A', '130912A', '140903A', '141212A', '150120A', '150423A',
          '150831A', '160303A', '160408A', '160410A', '160411A', '160601A', '160612A',
          '160624A', '161001A', '170112A', '170127B', '170428A', '170524A', '180715A',
          '180718A', '180727A', '180805B', '181126A', '190427A', '191031D', '200623A',
          '201221D'] # Rastinejad, et al.

In [4]:
sGRBs = swift[(swift["BAT T90[sec]"].apply(pd.to_numeric, errors="coerce") <= 2) | swift["GRB"].isin(GCN_flagged) | swift["GRB"].isin(published)].copy() # filter catalog for GRBs with a valid T90 that is <2s

In [5]:
# format columns
sGRBs.drop(['Time[UT]', 'BAT RA(J2000)', 'BAT Dec(J2000)', 'BAT 90%Error Radius[arcmin]',
            'BAT Fluence(15-150 keV)[10-7 erg/cm2]', 'BAT Fluence90% Error(15-150 keV)[10-7 erg/cm2]',
            'BAT 1-sec PeakPhoton Flux(15-150 keV)[ph/cm2/sec]', 'BAT 1-sec PeakPhoton Flux90% Error(15-150 keV)[ph/cm2/sec]',
            'BAT Photon Index(15-150 keV)(PL = simple power-law,CPL = cutoff power-law)',
            'BAT Photon Index90% Error(15-150 keV)', 'XRT 90%Error Radius[arcsec]', 'XRT Column Density(NH)[1021 cm-2]',
            'XRT Early Flux(0.3-10 keV)[10-11 erg/cm2/s]', 'XRT 11 Hour Flux(0.3-10 keV)[10-11 erg/cm2/s]',
            'XRT 24 Hour Flux(0.3-10 keV)[10-11 erg/cm2/s]', 'UVOT RA(J2000)', 'UVOT Dec(J2000)', 'UVOT 90%Error Radius[arcsec]',
            'Host Galaxy', 'Comments', 'References', 'Burst Advocate'],axis=1,inplace=True) # already in master catalog
sGRBs.rename(columns={'XRT Time to FirstObservation[sec]':'XRT dt [sec]','XRT InitialTemporalIndex':"α",
                      'XRTSpectral Index(Gamma)':'Γ','UVOT Time toFirst Observation[sec]':'UVOT dt [sec]',
                      'UVOT Other FilterMagnitudes':'Other UVOT Filters',"XRT RA(J2000)":"RA","XRT Dec(J2000)":"Dec"},inplace=True)

In [6]:
optical_obs = []
for idx,entry in sGRBs["Other UVOT Filters"].iteritems():
    if pd.isna(entry):
        entries = []
    else:
        entries = split_filters(entry)
    try:
        filterdict = dict([entry.split(">") if ">" in entry else entry.split("=") if "=" in entry else None for entry in entries])
        filterdict["idx"] = idx
        optical_obs.append(filterdict)
    except:
        print("error",idx,end="; ")

for obs in optical_obs:
    sGRBs.loc[obs["idx"], "Other UVOT Filters"] = [obs.copy()]

error 14; error 39; error 53; error 89; error 171; error 238; error 328; error 340; error 382; error 406; error 456; error 635; error 676; error 724; error 753; error 835; error 968; error 1033; error 1161; error 1163; error 1178; error 1230; error 1244; error 1397; error 1589; 

In [7]:
sGRBs["Redshift"] = ["".join([char for char in entry if char in numeric]) if type(entry) is str else entry for entry in sGRBs["Redshift"]]

## *Swift*-XRT lightcurves

In [8]:
xrt_data = pd.DataFrame(columns=['GRB', 'Time', 'Tpos', 'Tneg', 'Flux', 'Fluxpos', 'Fluxneg'])
for i,row in sGRBs.iterrows():
    GRB_ID = row["GRB"]
    print(GRB_ID+" "*(7-len(GRB_ID)),end=": ")
    mode = None
    try:
        Gamma,mode = get_photonIndex(GRB_ID,sGRBs)
        sGRBs.loc[i,"Beta_X"] = Gamma.value - 1
        sGRBs.loc[i,"Beta_X_neg"] = Gamma.minus
        sGRBs.loc[i,"Beta_X_pos"] = Gamma.plus
        print("index ✓",end=", ")
    except:
        print("index ✗",end=", ")
    try:
        fluxdata = XRT_lightcurve(GRB_ID,sGRBs)
        xrt_data = xrt_data.append(fluxdata,ignore_index=True)
        print("lightcurve ✓",end=" ")
    except:
        print("lightcurve ✗",end=" ")
    if mode=="WT":
        print("(used WT spectrum)")
    else:
        print()
        
# account for upper limits
xrt_data.loc[xrt_data["Fluxneg"]==0, "Fluxneg"] = np.inf

211227A: index ✓, lightcurve ✓ 
211221A: index ✓, lightcurve ✓ 
211207A: index ✓, lightcurve ✓ 
211106A: index ✓, lightcurve ✗ 
211023B: index ✓, lightcurve ✓ 
210919A: index ✓, lightcurve ✓ 
210726A: index ✓, lightcurve ✓ 
210725B: index ✓, lightcurve ✓ 
210708A: index ✓, lightcurve ✓ 
210704A: index ✓, lightcurve ✗ 
210618A: index ✓, lightcurve ✗ 
210413B: index ✗, lightcurve ✗ 
210410A: index ✓, lightcurve ✓ 
210323A: index ✓, lightcurve ✓ 
210217A: index ✓, lightcurve ✓ 
210119A: index ✗, lightcurve ✗ 
201221D: index ✓, lightcurve ✓ 
201221A: index ✓, lightcurve ✓ 
201015A: index ✓, lightcurve ✓ 
201006A: index ✓, lightcurve ✓ 
200917A: index ✓, lightcurve ✓ 
200907B: index ✓, lightcurve ✓ 
200826A: index ✓, lightcurve ✗ 
200729A: index ✓, lightcurve ✓ 
200716C: index ✓, lightcurve ✓ 
200623A: index ✗, lightcurve ✗ 
200522A: index ✓, lightcurve ✓ 
200517A: index ✓, lightcurve ✗ 
200512A: index ✓, lightcurve ✓ 
200411A: index ✓, lightcurve ✓ 
200409A: index ✓, lightcurve ✓ 
200219A:

In [13]:
sGRBs.to_csv("./products/Swift_sGRB_catalog.csv",index=False)
xrt_data.to_csv("./products/Swift_XRT_lightcurves.csv",index=False)